# 1. Getting Connected

__Copy the following function, from saskia, into your bash_profile:__


    function fnal_proxy {


    echo "MAIN DQM http://localhost:3333 --> g2be2:3333"
    echo "TEST DQM http://localhost:3334 --> g2be2:4999"
    echo "MAIN MIDAS : http://localhost:8080 --> g2be1:8080"
    echo "MAIN OPERATIONS : http://localhost:8888 --> g2sc:80"
    echo " "
    echo "FTS http://localhost:8787 --> g2be3:8787/fts/status"
    echo " "
    echo "CCC : http://localhost:6003 --> g2web:7000"
    echo "CCC-JSON : http://localhost:6004 --> g2web:7001"
    echo "CALO SLOW CONTROL http://localhost:4999 --> g2sc:5000"
    echo " "
    echo "G2TRACKER0 MIDAS : http://localhost:8181 --> g2tracker0:8080"
    echo "GM2LIV DQM CLIENT : http://localhost:3434 --> gm2liv-priv:3344"
    echo "TRACKER WEB MONITOR : http://localhost:5001 --> g2tracker0-priv:5002"
    echo " "
    echo "EFFICIENCY/POT : http://localhost:6002 --> g2web:5002"
    echo " "
    echo "ONLINE PSQL DB: psql -U gm2_writer -d gm2_online_prod -h localhost -p 5434"
    echo ""
    echo "SRV : http://localhost:8989 --> gm2-01.dhcp.fnal.gov:80"
    echo "G2DB : http://localhost:9002 --> g2db:9002"

    ssh -nNT -L 9005:192.168.30.65:80 -L 6004:g2web:7001 -L 6003:g2web:7000 -L 6002:g2web:5002 -L 4999:g2sc:500 -L 3332:g2field-be:8080 -L 3331:g2field-be:3335 -L 3333:g2be2:3333 -L 3334:g2be2:4999 -L 8787:g2be3:8787 -L 8080:g2be1:8080 -o GSSAPIAuthentication=yes G2Muon@g2gateway01.fnal.gov -L 8888:g2sc:80 -L 3434:gm2liv-priv:3344 -L 8181:g2tracker0-priv:8080 -L 5001:g2tracker0-priv:5002 -L 8989:gm2-01.dhcp.fnal.gov:80 -L 5434:192.168.30.51:5433 -L 9002:g2db:9002 -L 9003:192.168.22.79:80 -L 9004:192.168.22.79:81

    }

__in a terminal, connect to the database with:__

    fnal_proxy

__run psql (if not already installed, do so using brew)__

    psql -U gm2_reader -d gm2_online_prod -h localhost -p 5434


__Within gm2_online_prod, to display ctags and time, do:__

    select * from gm2ctag_dqm limit 10;
    
__You can order by time, for example, by doing__

    select * from gm2ctag_dqm order by time ASC limit 10;

__to quit psql, run__

    \q

# 3. Reading in from the psql database

In [46]:
import pandas as pd
import argparse,csv,datetime,time
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import matplotlib.dates as mdate
import psycopg2
from collections import defaultdict
import sys

### First get the column names for the three tables we're interested in

* gm2dq.ctagswithdqc
* gm2dq.subrun_time
* gm2dq.subrun_status

__We also may want to compare with the table:__

* gm2ctag_dqm

In [93]:
def connect(db):
    #connect to the database
    if (db ==  'localhost'):
        dsn  = "dbname=gm2_online_prod user=gm2_writer host=localhost port=5434"
    elif (db == 'g2db-priv'):
        dsn  = "dbname=gm2_online_prod user=gm2_writer host=g2db-priv port=5433"
    else:
        print ("None supported DB specified - run with --db=localhost or --db=g2db-priv")
        sys.exit(-1)
    conn = psycopg2.connect(dsn)
    curr = conn.cursor()
    return conn,curr

In [187]:
def get_labels(db='localhost', table='') :
    #connect
    conn,curr = connect(db)

    #get the column names for the specified table
    sql = "SELECT * FROM " + table +  " LIMIT 0"
    curr.execute(sql)
    colnames = [desc[0] for desc in curr.description]
    
    #close the connection without commiting any accidential changes
    conn.rollback()
    
    return colnames

In [188]:
print(get_labels(table='gm2dq.ctagswithdqc'), '\n \n')
print(get_labels(table='gm2dq.subrun_time'), '\n \n')
print(get_labels(table='gm2dq.subrun_status'), '\n \n')
print(get_labels(table='gm2ctag_dqm'))

['run', 'subrun', 'ctags', 't0val', 'fills'] 
 

['run', 'subrun', 'start_time', 'end_time', 'nevents', 'start_gps', 'end_gps'] 
 

['run', 'subrun', 'quad_condition', 'kicker_condition', 'quad_ok', 'ctags_ok', 'losses_ok', 'fillcuts_ok', 'field_ok', 'trolley_period', 'field_period', 'ctags_loose_ok', 'quad_loose_ok', 'ctags_repeat_ok', 'losses_repeat_ok', 'fillcuts_repeat_ok'] 
 

['time', 'ctags']


###  Get a list of runs/subruns for a subset of the 60 hour dataset

In [182]:
def get_runs(start, end, db='localhost', table='gm2dq.subrun_time'):
    #connect
    conn,curr = connect(db)
 
    #Prepare the sql command and execute
    what = f"select run, subrun, start_time, end_time from {table} "
    where = f"where start_time >= '{start}' and end_time <= '{end}' "
    order = "order by start_time ASC"
    sql = what + where + order
    curr.execute(sql)
    
    #fetch the data
    rows = curr.fetchall()
    
    #Close the connection without commiting any accidential changes
    conn.rollback()
    
    return rows

In [228]:
runs = get_runs(start="2018-04-22 00:00:00",end="2018-04-25 00:00:00")
df_time = pd.DataFrame.from_records(runs,columns=["run", "subrun", "start_time", "end_time"])
df_time.set_index(['run','subrun'],inplace=True)
df_time

start_time            end_time
run   subrun                                        
15906 16     2018-04-22 00:00:11 2018-04-22 00:00:20
      17     2018-04-22 00:00:23 2018-04-22 00:00:33
      18     2018-04-22 00:00:34 2018-04-22 00:00:44
      19     2018-04-22 00:00:45 2018-04-22 00:00:55
      20     2018-04-22 00:00:57 2018-04-22 00:01:07
...                          ...                 ...
15988 207    2018-04-24 23:59:26 2018-04-24 23:59:31
      208    2018-04-24 23:59:32 2018-04-24 23:59:37
      209    2018-04-24 23:59:39 2018-04-24 23:59:44
      210    2018-04-24 23:59:45 2018-04-24 23:59:50
      211    2018-04-24 23:59:51 2018-04-24 23:59:56

[31668 rows x 2 columns]

###  get the ctags for all the runs/subruns, 

In [229]:
def get_ctags(db='localhost', table='gm2dq.ctagswithdqc'):
    #connect
    conn,curr = connect(db)
 
    #Prepare the sql command and execute
    what = f"select run, subrun, ctags from {table} "
    order = "order by run,subrun ASC"
    sql = what + order
    curr.execute(sql)
    
    #fetch the data
    rows = curr.fetchall()
    
    #Close the connection without commiting any accidential changes
    conn.rollback()
    
    return rows






In [230]:
ctags = get_ctags()
df_ctag = pd.DataFrame.from_records(ctags,columns=["run", "subrun", "ctags"])
df_ctag.set_index(['run','subrun'],inplace=True)
df_ctag

ctags
run   subrun       
0     111     26599
      132     25746
      301         0
      326     24484
      427     26236
...             ...
17527 492     26689
      493     27301
      494     25219
      495     27586
      496     10812

[257510 rows x 1 columns]

In [237]:
df_total = pd.merge(df_time, df_ctag, on=['run','subrun'], how='inner')
df_total

start_time            end_time  ctags
run   subrun                                               
15921 0      2018-04-22 13:14:00 2018-04-22 13:14:34      0
      1      2018-04-22 13:14:36 2018-04-22 13:14:43      0
      2      2018-04-22 13:14:44 2018-04-22 13:14:52      0
      3      2018-04-22 13:14:53 2018-04-22 13:15:00      0
      4      2018-04-22 13:15:02 2018-04-22 13:15:19      0
...                          ...                 ...    ...
15988 207    2018-04-24 23:59:26 2018-04-24 23:59:31  21109
      208    2018-04-24 23:59:32 2018-04-24 23:59:37  21203
      209    2018-04-24 23:59:39 2018-04-24 23:59:44  21609
      210    2018-04-24 23:59:45 2018-04-24 23:59:50  21347
      211    2018-04-24 23:59:51 2018-04-24 23:59:56  21072

[26341 rows x 3 columns]

In [151]:
def get_ctags(start,end, db='localhost', table='gm2ctag_dqm') :

    conn,curr = connect(db)
    
    #get the ctags
    ctag_dict = {}
    sql = "select * from {} where time >= {} and time <= {} order by time DESC".format(table,start,end)
    sql = "select * from gm2ctag_dqm where time >= '%s' and time <= '%s' order by time DESC" % (start, end)
    sql = "select * from gm2ctag_dqm where time >= '{}' and time <= '{}' order by time DESC".format(start, end)
    sql = f"select * from gm2ctag_dqm where time >= '{start}' and time <= '{end}' order by time DESC"
    
#     sql = "select * from gm2dq.ctagswithdqc"
#     sql = "select * from gm2dq.subrun_time"
#     sql = "select * from gm2dq.subrun_status"
#     sql = "select * from " + var
    curr.execute(sql)
    conn.commit()
    rows = curr.fetchall()

    for row in rows:
        ct = int(row[1])
        ts = row[0]
        ctag_dict[ts] = ct

#     return ctag_dict
    return rows

In [152]:
get_ctags("2018-04-22 00:00:00", "2018-04-25 00:00:00",'localhost',table = "gm2ctag_dqm")


[(datetime.datetime(2018, 4, 24, 23, 59, 25), 542.5),
 (datetime.datetime(2018, 4, 24, 23, 58, 25), 540.2),
 (datetime.datetime(2018, 4, 24, 23, 57, 25), 549.9),
 (datetime.datetime(2018, 4, 24, 23, 56, 25), 542.0),
 (datetime.datetime(2018, 4, 24, 23, 55, 25), 552.6),
 (datetime.datetime(2018, 4, 24, 23, 54, 25), 547.1),
 (datetime.datetime(2018, 4, 24, 23, 53, 25), 535.4),
 (datetime.datetime(2018, 4, 24, 23, 52, 25), 558.7),
 (datetime.datetime(2018, 4, 24, 23, 51, 25), 538.8),
 (datetime.datetime(2018, 4, 24, 23, 50, 25), 532.6),
 (datetime.datetime(2018, 4, 24, 23, 49, 25), 531.0),
 (datetime.datetime(2018, 4, 24, 23, 48, 25), 525.7),
 (datetime.datetime(2018, 4, 24, 23, 47, 25), 529.1),
 (datetime.datetime(2018, 4, 24, 23, 46, 25), 518.6),
 (datetime.datetime(2018, 4, 24, 23, 45, 25), 531.5),
 (datetime.datetime(2018, 4, 24, 23, 44, 25), 546.7),
 (datetime.datetime(2018, 4, 24, 23, 43, 25), 525.3),
 (datetime.datetime(2018, 4, 24, 23, 42, 25), 548.5),
 (datetime.datetime(2018, 4,

In [57]:
times = get_ctags("2018-04-22 00:00:00", "2018-04-25 00:00:00",'localhost',var = "gm2dq.subrun_time")
times

[(14224,
  0,
  datetime.datetime(2018, 3, 27, 13, 10, 47),
  datetime.datetime(2018, 3, 27, 13, 11, 19),
  169,
  None,
  None),
 (14225,
  171,
  datetime.datetime(2018, 3, 27, 14, 22, 39),
  datetime.datetime(2018, 3, 27, 14, 22, 44),
  143,
  None,
  None),
 (11697,
  0,
  datetime.datetime(2018, 2, 12, 17, 57, 28),
  datetime.datetime(2018, 2, 12, 17, 58, 3),
  168,
  None,
  None),
 (11697,
  2,
  datetime.datetime(2018, 2, 12, 17, 58, 10),
  datetime.datetime(2018, 2, 12, 17, 58, 14),
  131,
  None,
  None),
 (11697,
  4,
  datetime.datetime(2018, 2, 12, 17, 58, 22),
  datetime.datetime(2018, 2, 12, 17, 58, 26),
  131,
  None,
  None),
 (11697,
  6,
  datetime.datetime(2018, 2, 12, 17, 58, 34),
  datetime.datetime(2018, 2, 12, 17, 58, 38),
  132,
  None,
  None),
 (11697,
  8,
  datetime.datetime(2018, 2, 12, 17, 58, 46),
  datetime.datetime(2018, 2, 12, 17, 58, 50),
  148,
  None,
  None),
 (11697,
  10,
  datetime.datetime(2018, 2, 12, 17, 58, 57),
  datetime.datetime(2018, 2,

In [58]:
status = get_ctags("2018-04-22 00:00:00", "2018-04-25 00:00:00",'localhost',var = "gm2dq.subrun_status")
status

[(17504,
  284,
  None,
  None,
  True,
  True,
  True,
  True,
  True,
  6,
  3,
  None,
  None,
  None,
  None,
  None),
 (17504,
  308,
  None,
  None,
  True,
  True,
  True,
  True,
  True,
  6,
  3,
  None,
  None,
  None,
  None,
  None),
 (15932,
  86,
  '10.1/15.3',
  '3.6/2.64/2.76',
  False,
  False,
  False,
  False,
  True,
  0,
  2,
  False,
  None,
  None,
  None,
  None),
 (15932,
  101,
  '10.1/15.3',
  '3.6/2.64/2.76',
  False,
  False,
  False,
  False,
  True,
  0,
  2,
  False,
  None,
  None,
  None,
  None),
 (15959,
  260,
  '13.1/18.3',
  '3.6/2.64/2.76',
  False,
  False,
  False,
  False,
  True,
  0,
  6,
  False,
  None,
  None,
  None,
  None),
 (15959,
  261,
  '13.1/18.3',
  '3.6/2.64/2.76',
  False,
  False,
  False,
  False,
  True,
  0,
  6,
  False,
  None,
  None,
  None,
  None),
 (15959,
  263,
  '13.1/18.3',
  '3.6/2.64/2.76',
  False,
  False,
  False,
  False,
  True,
  0,
  6,
  False,
  None,
  None,
  None,
  None),
 (15959,
  264,
  '13.1/1